In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [4]:
df = fetch_hourly_rides(12)

2025-05-09 11:41:20,698 INFO: Initializing external client
2025-05-09 11:41:20,699 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-09 11:41:22,620 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214627
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.85s) 


In [5]:
df

,pickup_hour,pickup_location_id,rides
0,2025-05-09 06:00:00+00:00,5282.02,0
1,2025-05-09 09:00:00+00:00,JC095,2
2,2025-05-09 00:00:00+00:00,HB302,0
3,2025-05-09 07:00:00+00:00,7260.09,0
4,2025-05-09 15:00:00+00:00,HB505,0
...,...,...,...
1559,2025-05-09 10:00:00+00:00,JC104,1
1560,2025-05-09 06:00:00+00:00,JC057,2
1561,2025-05-09 04:00:00+00:00,JC002,0
1562,2025-05-09 09:00:00+00:00,JC009,3


In [6]:
df_pred = fetch_predictions(12)

2025-05-09 11:41:29,354 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-09 11:41:29,370 INFO: Initializing external client
2025-05-09 11:41:29,371 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-09 11:41:30,917 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214627
2025-05-09 11:41:32,987 ERROR: No commits found for featuregroup: /apps/hive/warehouse/spring25_taxi_featurestore.db/citi_bike_hourly_model_prediction_1. Detail: Python exception: FlyingDuckException. gRPC client debug context: UNKNOWN:Error received from peer ipv4:51.79.26.27:5005 {created_time:"2025-05-09T15:41:32.8900039+00:00", grpc_status:2, grpc_message:"No commits found for featuregroup: /apps/hive/warehouse/spring25_taxi_featurestore.db/citi_bike_hourly_model_prediction_1. Detail: Python exception: FlyingDuckException"}. Client context: IOError: Server never sent a data message. Detail: Internal
Traceback (most rec

FeatureStoreException: No commits found for featuregroup: /apps/hive/warehouse/spring25_taxi_featurestore.db/citi_bike_hourly_model_prediction_1. Detail: Python exception: FlyingDuckException. gRPC client debug context: UNKNOWN:Error received from peer ipv4:51.79.26.27:5005 {created_time:"2025-05-09T15:41:32.8900039+00:00", grpc_status:2, grpc_message:"No commits found for featuregroup: /apps/hive/warehouse/spring25_taxi_featurestore.db/citi_bike_hourly_model_prediction_1. Detail: Python exception: FlyingDuckException"}. Client context: IOError: Server never sent a data message. Detail: Internal

In [ ]:
df_pred

In [ ]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [ ]:
merged_df

In [ ]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [ ]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

In [ ]:
merged_df

In [ ]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [ ]:
mae_by_hour["MAE"].mean()